In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
# Going to base this on: https://danijar.com/introduction-to-recurrent-networks-in-tensorflow/
# This just shows how the basic object behave, with a very simple sequence-to-sequence example
# We'll start with just learning the identity function on an alphabet of size 28

num_units = 10
num_layers = 1

seq_length = 25

cells = []
for _ in range(num_layers):
    cell = tf.contrib.rnn.BasicLSTMCell(num_units)  # Or LSTMCell(num_units)
    cells.append(cell)
cell = tf.contrib.rnn.MultiRNNCell(cells) # Stack the RNN into layers

num_symbols_in_alphabet = 5
num_output_classes = 5

data = tf.placeholder(tf.float32, [None, seq_length, num_symbols_in_alphabet], name = "data") # [batch size, string size, distinct symbols]
output, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32) # Outputs have the gates applied to m to produce h
W1 = tf.Variable(tf.random_uniform([num_units,num_output_classes],0,0.01))
output = tf.reshape(output, [-1, num_units])
predict = tf.nn.softmax(tf.matmul(output, W1))
predict = tf.reshape(predict, [-1, seq_length, num_output_classes])

target = tf.placeholder(tf.float32, [None, seq_length, num_output_classes], name = "target")

# loss = tf.losses.softmax_cross_entropy(tf.gather(target, indices = seq_length - 1, axis = 1), tf.gather(predict, indices = seq_length - 1, axis = 1))

loss = tf.losses.softmax_cross_entropy(target, predict)

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

rand_sample = [random.randint(0, num_symbols_in_alphabet-1) for i in range(seq_length)]
data_sample = [np.ndarray.tolist(np.identity(num_symbols_in_alphabet)[r:r+1][0]) for r in rand_sample]
target_sample = data_sample # Just trying to learn the identity
p1 = sess.run([predict], feed_dict = {data:[data_sample]})

print("Target")
t = np.argmax(target_sample, axis=1)
print(t)
print("\n\n")
print("First Prediction")
print(np.argmax(p1[0][0], axis=1))

print("\nTraining.....\n")
for i in range(10001):
    p2, _, l = sess.run([predict, updateModel, loss], feed_dict = {data:[data_sample] , target:[target_sample]})
    
    if i % 1000 == 0:
        print("New prediction after %i rounds of training" % i)
        k = np.argmax(p2[0], axis=1)
        print(k)
        print([k[q] == t[q] for q in range(len(k))])
        print("\tLoss %f" % l)

Target
[4 3 2 2 1 2 3 0 2 4 4 0 4 4 3 1 4 4 1 2 4 1 2 0 1]



First Prediction
[1 1 0 0 0 0 0 3 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3]
Training.....
New prediction after 0 rounds of training
[1 1 0 0 0 0 0 3 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, True, False, False, False]
	Loss 1.609432
New prediction after 1000 rounds of training
[4 4 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 2]
[True, False, True, True, False, True, False, False, True, True, True, False, True, True, False, False, True, True, False, False, True, False, True, False, False]
	Loss 1.435941
New prediction after 2000 rounds of training
[4 2 2 2 1 2 2 1 2 4 4 1 4 4 1 1 4 4 1 2 4 1 2 4 1]
[True, False, True, True, True, True, False, False, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, False, True]
	Loss 1.144555
New prediction after 3000 rou